In [ ]:
import pandas as pd
from collections import Counter
import json

# --- 1. VERİ YÜKLEME ---
# İşlenmiş diziler.jsonl dosyanızı yükleyin
file_path = "../data/processed/diziler.jsonl" 
data_list = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data_list.append(json.loads(line))

diziler_df = pd.DataFrame(data_list)
print(f"{len(diziler_df)} adet yazıt dizisi yüklendi.")
print(diziler_df.head())

# --- 2. SON İŞARET ANALİZİ (SUFFIX) ---
# 'dizi' sütununu kullandığınızdan eminiz (önceki çıktıya göre doğru)
terminal_signs = diziler_df['dizi'].apply(lambda x: x[-1] if (x and isinstance(x, list) and len(x) > 0) else None)

# 3. Terminal (Son) İşaret Frekansını Hesapla
terminal_counts = Counter(terminal_signs.dropna())
terminal_freq_df = pd.DataFrame(terminal_counts.items(), columns=['ngram', 'terminal_frekans'])
terminal_freq_df = terminal_freq_df.sort_values(by='terminal_frekans', ascending=False)

print("\n--- En Yaygın SON İşaretler (Suffix Adayları) ---")
print(terminal_freq_df.head())

# --- 4. GENEL FREKANS İLE KARŞILAŞTIRMA ---
try:
    # !!!!!!!!!!!!!!!!!! 1. DÜZELTME BURADA !!!!!!!!!!!!!!!!!!
    # Dosyayı 'figures' klasöründen DEĞİL, doğrudan 'reports' klasöründen oku
    general_freq_df = pd.read_csv("../reports/1_gram_frekans.csv")
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    # !!!!!!!!!!!!!!!!!! 2. DÜZELTME BURADA !!!!!!!!!!!!!!!!!!
    # CSV'den okunan 'ngram_1' (alt çizgili) sütun adını,
    # 'ngram' (alt çizgisiz) olarak DEĞİŞTİRİYORUZ.
    if 'ngram_1' in general_freq_df.columns:
        general_freq_df.rename(columns={'ngram_1': 'ngram'}, inplace=True)
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    # 5. İki frekansı birleştirme (merge)
    comparison_df = pd.merge(general_freq_df, terminal_freq_df, on='ngram', how='outer').fillna(0)
    
    # 'frekans' sütunu CSV'de doğru adlandırılmıştı (resme göre), bu satır çalışmalı.
    comparison_df = comparison_df[comparison_df['frekans'] > 0] 

    comparison_df['terminal_yuzdesi'] = (comparison_df['terminal_frekans'] / comparison_df['frekans']) * 100

    # 6. Sonek adaylarını bulma
    suffix_adaylari = comparison_df[comparison_df['terminal_yuzdesi'] > 50.0]
    suffix_adaylari = suffix_adaylari.sort_values(by='terminal_yuzdesi', ascending=False)

    print("\n--- Güçlü Sonek (Suffix) Adayları (Terminal % > 50) ---")
    print(suffix_adaylari)

    # 7. Sonuçları Kaydetme
    # Çıktı yolunun da 'reports' klasörüne (figures'in dışına) olduğundan emin ol
    output_path = "../reports/konumsal_analiz_suffix.csv"
    suffix_adaylari.to_csv(output_path, index=False)
    print(f"\nSuffix adayları raporu şuraya kaydedildi: {output_path}")

except FileNotFoundError:
    print("\nHATA: '../reports/1_gram_frekans.csv' dosyası bulunamadı.")
    print("LÜTFEN KONTROL EDİN: Terminalden 'python src/analysis/stats.py' komutunu çalıştırdınız mı?")
except KeyError as e:
    # Hata devam ederse hangi sütunu bulamadığını net olarak yazdır
    print(f"\nHATA: DataFrame'de bir sütun bulunamadı (KeyError): {e}")
    print("\nLütfen 'general_freq_df' (1_gram_frekans.csv'den) veya 'terminal_freq_df' ('diziler.jsonl'den) sütun adlarını kontrol edin.")